In [ ]:
import pandas as pd

start_date = 1641168000
end_date = 1642550400

specific_columns = ['Date', 'Close', 'Volume']

tickers = ['MSFT', 'AAPL', 'IBM']

all_data = pd.DataFrame()

for ticker in tickers:
    query_string = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={start_date}&period2={end_date}&interval=1d&events=history&includeAdjustedClose=true'
    df = pd.read_csv(query_string, usecols=specific_columns)
    df['Symbol'] = ticker

    all_data = all_data.append(df, ignore_index=True)

all_data.to_csv('stock_data.csv', index=False)

print("CSV file created successfully!")



In [ ]:
import pandas as pd

df = pd.read_csv("stock_data.csv")


df['Date'] = pd.to_datetime(df['Date'])


# Calculate 2-day moving average
df['2-Day Moving Average'] = df['Close'].rolling(window=2, min_periods=1).mean()

# Calculate 2-day VWAP
df['2-Day VWAP'] = (df['Close'] * df['Volume']).rolling(window=2, min_periods=1).sum() / df['Volume'].rolling(window=2, min_periods=1).sum()


pivot_table = df.pivot_table(index='Date', columns='Symbol', values=['Close', '2-Day Moving Average'])


pivot_table.columns = [f'{col[1]} {col[0]}' for col in pivot_table.columns]


df.to_csv("Stock_ClosePx.csv", index=False)

print("CSV File Created successfully")

In [ ]:
from flask import Flask, render_template
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
import os

current_directory = os.getcwd()
print("Current Directory", current_directory)

file_path = '/Users/devanshidesai/Downloads/Problem2/data/stock_CloseVWAP.csv'

df = pd.read_csv(file_path)

plt.figure(figsize=(10, 6))

# Plot VWAP for IBM
plt.plot(df[df['Symbol'] == 'IBM']['Date'], df[df['Symbol'] == 'IBM']['2-Day VWAP'], label='IBM')

# Plot VWAP for AAPL
plt.plot(df[df['Symbol'] == 'AAPL']['Date'], df[df['Symbol'] == 'AAPL']['2-Day VWAP'], label='AAPL')

# Plot VWAP for MSFT
plt.plot(df[df['Symbol'] == 'MSFT']['Date'], df[df['Symbol'] == 'MSFT']['2-Day VWAP'], label='MSFT')


# Customize the plot
plt.xlabel('Date')
plt.ylabel('2-Day VWAP')
plt.title('2-Day VWAP Over Time for IBM, AAPL, MSFT')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()


In [ ]:
import pandas_datareader.data as web
import dash
import dash_core_components
import dash_html_components
from dash.dependencies import Input, Output
import datetime


app = dash.Dash()
app.title = "Stock Visualization"
app.layout = html.Div(children = [
    html.H1('Stock Visualization Dashboard'),
    html.H4('Please enter the stock name'),
    dcc.Input(id="input", value='', type='text'),
    html.Div(id="output-graph")
])

@app.callback (
    Output(component_id = "output-graph",
    component_property = 'children'),
    [Input(component_id = "input",
    component_property = "value")]       
)
def update_value(input_data):
    start = datetime.datetime(2022, 3, 3)
    end = datetime.datetime(2022, 1, 19)
    df = web.DataReader(input_data, 'yahoo',  start, end)
    return dcc.Graph(id="demo", figure={'data': [
        {'x': df.index, 'y':df.Close, 'type': 'line', 'name': input_data}], 'layout': {'title': input_data}})
    
if  __name__ == '__main__':
    app.run_server(debug=True)